# Library

In [1]:
import cv2 as cv
import numpy as np
from cvzone.FaceDetectionModule import FaceDetector
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.HandTrackingModule import HandDetector

-------------

# Eye Landmark

In [2]:
LEFT_EYE=[362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
RIGHT_EYE= [155, 154, 153, 145, 144, 163, 7, 33, 246, 161, 160, 159, 158, 159, 173, 133]

# Facedetector object

In [3]:
face_detector= FaceDetector()
meshdetector= FaceMeshDetector(maxFaces=1)
hands_detector= HandDetector(detectionCon = 0.5, maxHands=2)

I0000 00:00:1740231350.674320  175487 gl_context.cc:369] GL version: 2.1 (2.1 INTEL-10.4.14), renderer: Intel HD Graphics 3000 OpenGL Engine
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1740231350.693157  175796 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1740231350.738140  175487 gl_context.cc:369] GL version: 2.1 (2.1 INTEL-10.4.14), renderer: Intel HD Graphics 3000 OpenGL Engine
W0000 00:00:1740231350.761288  175807 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740231350.791933  175809 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1740231350.809041  175487 gl_context.cc:369] GL version: 2.1 (2.1 INTEL-10.4.14), renderer: Intel HD Gra

-------------------

# Select video file or webcam

In [4]:
webcam= True
video_path= 'video.mp4'

In [5]:
if webcam:
    cap= cv.VideoCapture(0)
else:
    cap= cv.VideoCapture(video_path)

if (cap.isOpened()== False):
    print("Error opening video stream or file")

W0000 00:00:1740231350.957029  175817 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740231351.030751  175817 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


# Run program

In [6]:
while cap.isOpened():
    ret, frame= cap.read()
    if not ret:
        break
    
    frame_eye= frame.copy()
    face_img, bbox= face_detector.findFaces(frame_eye)
    face_img, faces= meshdetector.findFaceMesh(frame_eye)
    hand, frame= hands_detector.findHands(frame)

    if hand:
        hand1= hand[0]
        lmlist1= hand1["lmList"]
        length, info, frame= hands_detector.findDistance(lmlist1[8][:-1], lmlist1[4][:-1], frame)
        if len(hand)> 1:
            hand2= hand[1]
            lmlist2= hand2["lmList"]
            length, info, frame= hands_detector.findDistance(lmlist2[8][:-1], lmlist2[4][:-1], frame)
 
    if bbox:
        center = bbox[0]["center"]
        if faces:
            text= [" ", " "]
            for i, eye in enumerate([LEFT_EYE, RIGHT_EYE]):
                eye_points = np.array([[faces[0][p][0], faces[0][p][1]] for p in eye])
                (ex, ey, ew, eh) = cv.boundingRect(eye_points)
                eye_roi = frame_eye[ey:ey+eh, ex:ex+ew]
                eye_roi_gr = cv.cvtColor(eye_roi, cv.COLOR_BGR2GRAY)
                _, iris = cv.threshold(eye_roi_gr, 40, 255, cv.THRESH_BINARY_INV)
                contours, _ = cv.findContours(iris, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
                contours = sorted(contours, key=lambda x: cv.contourArea(x), reverse=True)

                if contours:
                    (ix, iy, iw, ih) = cv.boundingRect(contours[0])
                    ix_cntr, iy_centr = ix + int(iw/2) + ex, iy + int(ih/2) + ey
                    cv.circle(frame, (ix_cntr, iy_centr), 5, (0, 255, 0), -1)
                    
                    ix_cntr_e = ix + int(iw/2)
                    if ix_cntr_e > int(3*ew/5):
                        text[i]= "left"
                    elif ix_cntr_e < int(2*ew/5):
                        text[i]= "right"
                    else:
                        text[i]= "center"

            text= f"Left eye: {text[0]}, Right eye: {text[1]}"
            (w, h), _ = cv.getTextSize(text, cv.FONT_HERSHEY_PLAIN, 3, 2)
            cv.rectangle(frame, (100, 100 - h), (100 + w, 100 + h), (0, 60, 0), -1)
            cv.putText(frame, text, (100, 100), cv.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 2)

    cv.imshow('Combined Tracking', frame)

    if cv.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


W0000 00:00:1740231353.329492  175809 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
